In [129]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import kaggle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import featuretools as ft
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer, MultiLabelBinarizer
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import NearMiss
from numpy import where


import os
import composeml as cp
import featuretools as ft
import dask.dataframe as dd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
from dask.distributed import Client

My project is focused on predicitng wether an item will be purchased or removed from the cart on an ecomemrce store. This will be used to help in marketing, sales. On the supply chain side it will also give more insights into how a product is viewed by customers and how much a brand/product should be stocked or not.

### Importing and preparing the data

In [130]:
#Convert csv to DataFrame
oct19 = pd.read_csv("../data/2019-Oct.csv")
nov19 = pd.read_csv("../data/2019-Nov.csv")
dec19 = pd.read_csv("../data/2019-Dec.csv")
jan20 = pd.read_csv("../data/2020-Jan.csv")
feb20 = pd.read_csv("../data/2020-Feb.csv")

In [131]:
print(oct19.shape)
print(nov19.shape)
print(dec19.shape)
print(jan20.shape)
print(feb20.shape)
print(oct19.shape[0]+nov19.shape[0]+dec19.shape[0]+jan20.shape[0]+feb20.shape[0])

(4102283, 9)
(4635837, 9)
(3533286, 9)
(4264752, 9)
(4156682, 9)
20692840


In [132]:
ecom_df = pd.concat([oct19, nov19, dec19, jan20, feb20], axis=0)
ecom_df.head()
print(ecom_df.shape)

(20692840, 9)


In [133]:
ecom_df = ecom_df[ecom_df.event_type != 'view']
ecom_df = ecom_df[ecom_df.event_type != 'cart']

# Data Cleaning

### NAs and how to deal with them

In [134]:
#8757117 nas is Brand
#4598 nas in user_session
#20339246 nas in category_code DROPPED

In [135]:
#NAs in Brand
ecom_df.brand.fillna(value="unknown", axis=0, inplace=True)

In [136]:
#NAs in user_session 
ecom_df.dropna(inplace=True)

In [137]:
ecom_df.category_code.isna().sum()

0

In [138]:
ecom_df.category_code.value_counts()

appliances.environment.vacuum             16225
stationery.cartrige                       15751
apparel.glove                             15683
furniture.bathroom.bath                    4086
furniture.living_room.cabinet              2046
accessories.bag                             754
accessories.cosmetic_bag                    500
appliances.personal.hair_cutter             276
appliances.personal.massager                212
appliances.environment.air_conditioner      104
furniture.living_room.chair                  26
Name: category_code, dtype: int64

In [139]:
#Drop this column, as we will use category code instead
ecom_df.drop("category_id", axis=1, inplace = True)

In [140]:
#Get rid of negative prices
ecom_df = ecom_df[ecom_df['price']>= 0]

In [141]:
#Set target to binary
ecom_df.event_type = ecom_df["event_type"].replace({'purchase': 1, 'remove_from_cart': 0})

# Feature Engineering

purchase per session, group session, total of price added up for each purchase in session

### Time FE

In [142]:
ecom_df['event_time'] = pd.to_datetime(ecom_df['event_time'])

In [143]:
ecom_df['month'] = pd.DatetimeIndex(ecom_df['event_time']).month
ecom_df['2020'] = pd.DatetimeIndex(ecom_df['event_time']).year
ecom_df['hour'] = pd.DatetimeIndex(ecom_df['event_time']).hour
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,2020,hour
264,2019-10-01 00:07:58+00:00,0,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
269,2019-10-01 00:08:03+00:00,0,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
274,2019-10-01 00:08:08+00:00,0,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
276,2019-10-01 00:08:10+00:00,0,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
278,2019-10-01 00:08:18+00:00,0,5775822,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0


In [144]:
#convert year to binary
ecom_df['2020'] = ecom_df['2020'].replace({2019: 0, 2020: 1})

In [145]:
ecom_df['day'] = ecom_df['event_time'].dt.dayofweek
ecom_df['day'] = ecom_df['day'].replace({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})

In [146]:
when_df = pd.DataFrame({'hour':range(1, 25)})
bins = [0,6,12,18,24]
labels = ['Night', 'Morning','Afternoon','Evening']
when_df['when'] = pd.cut(when_df['hour'], bins=bins, labels=labels, include_lowest=True)
#fix bellow
ecom_df = ecom_df.join(when_df['when'], on="hour")

In [147]:
ecom_df['month'] = ecom_df['month'].replace({1: 'January', 2: 'Febuary', 10: 'October', 11: 'November', 12: 'December'})

In [148]:
ecom_df.sample()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,2020,hour,day,when
1780529,2020-01-15 00:45:10+00:00,0,5756537,apparel.glove,unknown,6.19,561013339,13fd1a20-607f-4989-98b7-c141787c5cf3,January,1,0,Wednesday,Night


In [24]:
#ecom_df["current_session_clicks"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

In [25]:
#need to turn the features into numeric for bellow to work
#ecom_df["current_session_event_types"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

# Labeling

In [26]:
#encoder = LabelBinarizer()
#etype_labels = pd.DataFrame(encoder.fit_transform(ecom_df['event_type']), columns=encoder.classes_, index=ecom_df.index)
#etype_labels.head()

In [149]:
encoder = LabelBinarizer()
brand_labels = pd.DataFrame(encoder.fit_transform(ecom_df['brand']), columns=encoder.classes_, index=ecom_df.index)
brand_labels.head()

,babyliss,benovy,cutrin,depilflax,dewal,domix,emil,farmavita,farmona,gehwol,...,nitrimax,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
276,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
encoder = LabelBinarizer()
productid_labels = pd.DataFrame(encoder.fit_transform(ecom_df['product_id']), columns=encoder.classes_, index=ecom_df.index)
productid_labels.head()

,4923,4962,4965,4986,5395,6736,6737,6817,8098,8372,...,5926444,5926445,5926580,5926583,5926585,5926586,5928359,5929649,5929650,5929651
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
ohe = OneHotEncoder(sparse=False, drop='first')
catco_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['category_code']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["category_code"]].columns))
catco_labels.head()

,category_code_accessories.cosmetic_bag,category_code_apparel.glove,category_code_appliances.environment.air_conditioner,category_code_appliances.environment.vacuum,category_code_appliances.personal.hair_cutter,category_code_appliances.personal.massager,category_code_furniture.bathroom.bath,category_code_furniture.living_room.cabinet,category_code_furniture.living_room.chair,category_code_stationery.cartrige
264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [152]:
ohe = OneHotEncoder(sparse=False, drop='first')
day_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['day']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["day"]].columns))
day_labels.head()

,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
264,0.0,0.0,0.0,0.0,1.0,0.0
269,0.0,0.0,0.0,0.0,1.0,0.0
274,0.0,0.0,0.0,0.0,1.0,0.0
276,0.0,0.0,0.0,0.0,1.0,0.0
278,0.0,0.0,0.0,0.0,1.0,0.0


In [153]:
ohe = OneHotEncoder(sparse=False, drop='first')
month_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['month']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["month"]].columns))
month_labels.head()

,month_Febuary,month_January,month_November,month_October
264,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,1.0
278,0.0,0.0,0.0,1.0


In [154]:
ohe = OneHotEncoder(sparse=False, drop='first')
when_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['when']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["when"]].columns))
when_labels.head()

,when_Evening,when_Morning,when_Night
264,0.0,0.0,1.0
269,0.0,0.0,1.0
274,0.0,0.0,1.0
276,0.0,0.0,1.0
278,0.0,0.0,1.0


In [155]:
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,2020,hour,day,when
264,2019-10-01 00:07:58+00:00,0,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,0,0,Tuesday,Night
269,2019-10-01 00:08:03+00:00,0,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,0,0,Tuesday,Night
274,2019-10-01 00:08:08+00:00,0,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,0,0,Tuesday,Night
276,2019-10-01 00:08:10+00:00,0,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,0,0,Tuesday,Night
278,2019-10-01 00:08:18+00:00,0,5775822,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,0,0,Tuesday,Night


In [156]:
#Join our Labeled Columns to our main DataFrame
#ecom_df = ecom_df.join(etype_labels)
ecom_df = ecom_df.join(brand_labels)
ecom_df = ecom_df.join(productid_labels)
ecom_df = ecom_df.join(catco_labels)
ecom_df = ecom_df.join(day_labels)
ecom_df = ecom_df.join(month_labels)
ecom_df = ecom_df.join(when_labels)                        
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,2020,...,day_Thursday,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October,when_Evening,when_Morning,when_Night
66,2019-11-01 00:04:33+00:00,1,5766980,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
67,2019-11-01 00:04:33+00:00,1,5767494,stationery.cartrige,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
68,2019-11-01 00:04:33+00:00,1,5775813,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
69,2019-11-01 00:04:33+00:00,1,5775814,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
264,2019-10-01 00:07:58+00:00,0,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


# Preprocessing Data

### Train Test Split

In [157]:
X = ecom_df.drop(["month", "hour", "day", "when", "user_id", "product_id", "category_code", "event_time", "event_type", "brand", "user_session"], axis=1)
y = ecom_df.event_type

In [158]:
y.value_counts()

0    65903
1    27518
Name: event_type, dtype: int64

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Standard Scalar

In [ ]:
ss = StandardScaler()
X_train_ss = pd.DataFrame(ss.fit_transform(X_train), index=X_train.index, columns=X_train.columns)  

In [ ]:
ss = StandardScaler()
X_test_ss = pd.DataFrame(ss.fit_transform(X_test), index=X_test.index, columns=X_test.columns)

### SMOTE

In [42]:
sm = SMOTE(random_state=42)
X_tr_sm, y_tr_sm = sm.fit_resample(X_train_ss, y_train)

In [74]:
# summarize class distribution
counter = Counter(y)
print(counter)
# define the undersampling method
undersample = NearMiss(version=3, n_neighbors=3)
# transform the dataset
X, y = undersample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({'purchase': 38704, 'remove_from_cart': 28081})


/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:177: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(


Counter({'remove_from_cart': 28081, 'purchase': 20654})


# Models

### LGBMClassifier

In [ ]:
import lightgbm as lgb

In [ ]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train_ss,  y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train_ss)
test_preds_lgbm = lgbm.predict(X_test_ss)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

In [ ]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train,  y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train)
test_preds_lgbm = lgbm.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

In [ ]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_tr_sm,  y_tr_sm)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_tr_sm)
test_preds_lgbm = lgbm.predict(X_test_ss)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

### RandomForest

In [160]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

# Predict on training and test sets
training_preds_rf = rf.predict(X_train)
test_preds_rf = rf.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf)
print(first_results)

              precision    recall  f1-score   support

           0       0.82      0.90      0.86     13175
           1       0.69      0.53      0.60      5510

    accuracy                           0.79     18685
   macro avg       0.76      0.71      0.73     18685
weighted avg       0.78      0.79      0.78     18685



In [161]:
rf2 = RandomForestClassifier(class_weight='balanced')

rf2.fit(X_train, y_train)

# Predict on training and test sets
training_preds_rf2 = rf2.predict(X_train)
test_preds_rf2 = rf2.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf2)
print(first_results)

              precision    recall  f1-score   support

           0       0.84      0.81      0.83     13175
           1       0.59      0.63      0.61      5510

    accuracy                           0.76     18685
   macro avg       0.71      0.72      0.72     18685
weighted avg       0.76      0.76      0.76     18685



### DecisionTreeClassifier

In [162]:
tree_clf = DecisionTreeClassifier() 

tree_clf.fit(X_train, y_train)

# Predict on training and test sets
training_preds_tclf = tree_clf.predict(X_train)
test_preds_tclf = tree_clf.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_tclf)
print(first_results)

              precision    recall  f1-score   support

           0       0.82      0.91      0.86     13175
           1       0.71      0.52      0.60      5510

    accuracy                           0.79     18685
   macro avg       0.76      0.71      0.73     18685
weighted avg       0.78      0.79      0.78     18685



In [163]:
tree_clf2 = DecisionTreeClassifier(class_weight='balanced') 

tree_clf2.fit(X_train, y_train)

# Predict on training and test sets
training_preds_tclf2 = tree_clf2.predict(X_train)
test_preds_tclf2 = tree_clf2.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_tclf2)
print(first_results)

              precision    recall  f1-score   support

           0       0.85      0.76      0.81     13175
           1       0.55      0.69      0.61      5510

    accuracy                           0.74     18685
   macro avg       0.70      0.73      0.71     18685
weighted avg       0.76      0.74      0.75     18685



In [168]:
y_test

3720468    1
3839663    0
2110095    0
1168201    0
899787     1
          ..
1052252    1
3078970    0
622452     0
2472169    0
1774235    0
Name: event_type, Length: 18685, dtype: int64

# Visuals

In [176]:
#confusion matrix
plot_confusion_matrix(estimator=rf2, y_true=y_train, X=X_train)
plt.savefig("conufsion_matrix_train")

### Graph 1

In [2]:
user_df = ecom_df[['user_id','event_type','price']]

NameError: name 'ecom_df' is not defined

In [ ]:
graph_df = user_df[['user_id','price']]
graph_df = graph_df.groupby(['user_id'])['price'].sum().reset_index()
graph_df = graph_df.sort_values(by=['price'],ascending=False)
graph_df = graph_df.rename(columns={'price':'spent'})

In [ ]:
price_list = graph_df['spent'].to_list()
plt.hist(x= price_list,bins=100,log=True, color='green')
plt.xlabel('Spent amount')
plt.ylabel('Customer Count')
plt.title('Customer Spending')
plt.savefig("Customer_Spend")

### Graph 2

In [40]:
# Daily visitors number
fig, ax = plt.subplots(figsize=(15,7))
date = ecom_df.loc[:,['event_time','user_id']]
visitor_by_date = date.groupby(['event_time'])['user_id'].nunique()
visitor_by_date.plot(ax=ax)
plt.savefig("test")

### Graph 3

In [214]:
ax = sns.countplot(x=ecom_df["event_type"], data=ecom_df)
plt.savefig("Target_Value_Counts")

In [213]:
ecom_df["event_type"].value_counts()

0    65903
1    27518
Name: event_type, dtype: int64

### Graph 4

In [79]:
cart_count = ecom_df.groupby(["event_type"])["brand"].value_counts()
#({'purchase': 1, 'remove_from_cart': 0})

In [94]:
cart_p = cart_count[1]
cart_r = cart_count[0]
cart_p

brand
italwax      6845
unknown      6111
benovy       2999
max          1774
polarus      1347
depilflax    1244
nitrimax     1129
emil         1100
runail       1096
domix         928
irisk         809
nitrile       657
jessnail      441
yoko          292
gezatone      169
dewal         122
milv          119
farmona        78
profepil       62
vosev          33
gehwol         26
shik           26
nirvel         24
mavala         22
kinetics       16
naturmed       14
naomi          11
jaguar          7
cutrin          6
farmavita       4
kondor          4
kosmekka        2
masura          1
Name: brand, dtype: int64

In [128]:
cart_p = cart_count[1]
cart_r = cart_count[0]
cart_fin = pd.concat([cart_p, cart_r], axis=1)
cart_fin.columns.values[0]="Purchase"
cart_fin.columns.values[1]="Removed"
cart_fin = cart_fin.fillna(0.0)
cart_fin["Ratio"] = cart_fin.Purchase/cart_fin.Removed
cart_fin.sort_values(by="Ratio")

,Purchase,Removed,Ratio
shifei,0.0,1,0.000000
babyliss,0.0,14,0.000000
oniq,0.0,6,0.000000
kosmekka,2.0,106,0.018868
naomi,11.0,115,0.095652
vosev,33.0,280,0.117857
shik,26.0,191,0.136126
jessnail,441.0,2295,0.192157
irisk,809.0,3459,0.233883
profepil,62.0,231,0.268398


In [123]:
ax = cart_fin.plot.bar(y='Ratio', rot=0)
plt.xlabel('Brand')
plt.ylabel('Ratio P/R')
plt.xticks(rotation=90)
plt.savefig("Cart_Ratio")

### Graph ?

In [64]:
cart_count.plot.bar(figsize=(7, 5))
plt.legend(title='Value Counts', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Count')
plt.ylabel('Brand')
plt.savefig("Brand_Behaviour")

In [ ]:
sns.barplot(x='no_employees', y='Count', data=dft, hue='treatment')

In [41]:
plt.hist([bank[0], bank[1]], color=['r','b'], alpha=0.5)
plt.savefig("Brand_Behaviour")

In [ ]:
df_gb = df.groupby(['year', 'month', 'class']).size().unstack(level=2)

### Graph 5

In [66]:
grp_by_hr_event_type = ecom_df.groupby(['hour','event_type']).count()

In [68]:
layout= dict(title="Hourly Store Traffic", xaxis_title="Time of Day", yaxis_title="Number of  Users")
grp_by_hr_event_type['user_id'].unstack(1).plot(kind="bar", layout=layout)
plt.savefig("test")

In [70]:
grp_by_brand_count = ecom_df.groupby(['brand','event_type']).count()

In [72]:
layout= dict(title="Hourly Store Traffic", xaxis_title="Time of Day", yaxis_title="Number of  Users")
grp_by_brand_count['user_id'].unstack(1).plot(kind="bar", layout=layout, figsize=(16,24))
plt.savefig("test2")